### Load Data

In [1]:
import pandas as pd

In [2]:
links = pd.read_csv('/Users/gabrielwarner/Downloads/ml-latest-small/links.csv')
links

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9737,193581,5476944,432131.0
9738,193583,5914996,445030.0
9739,193585,6397426,479308.0
9740,193587,8391976,483455.0


In [3]:
movies = pd.read_csv('/Users/gabrielwarner/Downloads/ml-latest-small/movies.csv')

In [4]:
ratings = pd.read_csv('/Users/gabrielwarner/Downloads/ml-latest-small/ratings.csv')


In [5]:
tags = pd.read_csv('/Users/gabrielwarner/Downloads/ml-latest-small/tags.csv')

In [6]:
ratings = ratings.drop('timestamp', axis=1)

In [7]:
ratings.shape

(100836, 3)

In [8]:
new = pd.merge(links, movies, on='movieId')

In [9]:
new_2 = pd.merge(new, ratings, on='movieId')

In [10]:
new_3 = pd.merge(new_2, tags, on='movieId')

### Merge Data Frames

In [11]:
# merge links and movie
#frames_1 = [links, movies]
#new = pd.concat(frames_1)


In [12]:
# merge new and ratings
#frames_2 = [new, ratings]
#new_2 = pd.concat(frames_2)

In [13]:
# Drop movieId and timestamp
#tags_2 = tags.drop(columns=['movieId', 'timestamp'])

In [14]:
# concat new_2 and tags
#frames = [new_2, tags_2]
#new_3 = pd.concat(frames)
#new_3 = pd.merge(new_2, tags)


In [15]:
new_3.shape

(233213, 10)

### Preprocessing

In [16]:
# Check for null values
new_3.isnull().sum()

movieId      0
imdbId       0
tmdbId       0
title        0
genres       0
userId_x     0
rating       0
userId_y     0
tag          0
timestamp    0
dtype: int64

In [17]:
new_3['movieId'].nunique()

1554

In [18]:
# replace | with , in the genres column
new_3['genres'] = new_3['genres'].str.replace('|',' , ')

In [19]:
#df = new_3.drop_duplicates(subset=['movieId'])

In [20]:
# convert df to a csv file
new_3.to_csv('/Users/gabrielwarner/Data-Science/Projects/phase_4/movies_2.csv')

### Explode genres column

In [21]:
# split genre column based on ,
new_3['new_genres'] = new_3['genres'].str.split(',', 10)

In [22]:
# explode new_genres
df = new_3.explode('new_genres')

In [23]:
# drop genres from df
df_2 = df.drop(columns=['genres'])

In [24]:
df_2.head()

,movieId,imdbId,tmdbId,title,userId_x,rating,userId_y,tag,timestamp,new_genres
0,1,114709,862.0,Toy Story (1995),1,4.0,336,pixar,1139045764,Adventure
0,1,114709,862.0,Toy Story (1995),1,4.0,336,pixar,1139045764,Animation
0,1,114709,862.0,Toy Story (1995),1,4.0,336,pixar,1139045764,Children
0,1,114709,862.0,Toy Story (1995),1,4.0,336,pixar,1139045764,Comedy
0,1,114709,862.0,Toy Story (1995),1,4.0,336,pixar,1139045764,Fantasy


### OHE title

In [25]:
one_hot = pd.get_dummies(df_2['title'])

In [26]:
df = df_2.drop('title',axis = 1)

In [27]:
df = pd.concat([df, one_hot], axis=1)

In [28]:
df.shape

(765271, 1563)

In [29]:
one_hot.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 765271 entries, 0 to 233212
Columns: 1554 entries, (500) Days of Summer (2009) to eXistenZ (1999)
dtypes: uint8(1554)
memory usage: 1.1 GB


### OHE tag

In [30]:
one_hot_tag = pd.get_dummies(df['tag'])

In [31]:
df = df.drop('tag',axis = 1)

In [32]:
df = pd.concat([df, one_hot_tag], axis=1)

In [33]:
one_hot_tag.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 765271 entries, 0 to 233212
Columns: 1584 entries, "artsy" to zombies
dtypes: uint8(1584)
memory usage: 1.1 GB


### OHE new_genres

In [34]:
one_hot_genre = pd.get_dummies(df['new_genres'])

In [35]:
df =df.drop('new_genres',axis = 1)

In [36]:
df = pd.concat([df, one_hot_genre], axis=1)

### Base Line Model

In [37]:
from sklearn.neighbors import NearestNeighbors

In [38]:
#knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)
#knn.fit(df)

### Suprise

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 765271 entries, 0 to 233212
Columns: 3209 entries, movieId to Western
dtypes: float64(2), int64(5), uint8(3202)
memory usage: 2.3 GB


### Change Data Frames to suprise DataFrame

In [40]:
import surprise

In [41]:
from surprise.model_selection import train_test_split
# from sklearn.model_selection import train_test_split

In [42]:
ratings.rating.describe()

count    100836.000000
mean          3.501557
std           1.042529
min           0.500000
25%           3.000000
50%           3.500000
75%           4.000000
max           5.000000
Name: rating, dtype: float64

In [43]:
ratings.shape

(100836, 3)

In [44]:
reader = surprise.Reader(rating_scale = (4., 5.))

In [45]:
tags

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
...,...,...,...,...
3678,606,7382,for katie,1171234019
3679,606,7936,austere,1173392334
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodshed,1493843978


In [46]:
data = surprise.Dataset.load_from_df(ratings, reader)

In [47]:
df.head()

,movieId,imdbId,tmdbId,userId_x,rating,userId_y,timestamp,(500) Days of Summer (2009),...And Justice for All (1979),10 Cloverfield Lane (2016),...,Musical,Musical,Mystery,Mystery,Romance,Romance,Sci-Fi,Sci-Fi,Thriller,Western
0,1,114709,862.0,1,4.0,336,1139045764,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,1,114709,862.0,1,4.0,336,1139045764,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,1,114709,862.0,1,4.0,336,1139045764,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,1,114709,862.0,1,4.0,336,1139045764,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,1,114709,862.0,1,4.0,336,1139045764,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [49]:
# from sklearn.model_selection import train_test_split

In [50]:
trainset, testset = train_test_split(data, test_size=0.2)

In [51]:
#trainset.head()

### KNNS Basic (Best RMSE=.5)

In [52]:
from surprise.prediction_algorithms import knns
from surprise.similarities import cosine, msd, pearson
from surprise import accuracy

In [53]:
basic = knns.KNNBasic()
basic.fit(trainset)


Computing the msd similarity matrix...
Done computing similarity matrix.


In [54]:
predictions = basic.test(testset)

In [55]:
print(accuracy.rmse(predictions))

RMSE: 1.1491
1.1490831090848619


In [56]:
sim_pearson = {'name':'pearson', 'user_based':False}
knn_means = knns.KNNWithMeans(sim_options=sim_pearson)
knn_means.fit(trainset)
predictions = knn_means.test(testset)
print(accuracy.rmse(predictions))

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 1.1477
1.1477421480747632


### Matrix Factorization

In [57]:
from surprise.prediction_algorithms import SVD
from surprise.model_selection import GridSearchCV

### Suprise with grid search

In [58]:
from sklearn.neighbors import KNeighborsClassifier

In [59]:
param_grid = {'n_factors':[20, 50, 75, 100],'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
               'reg_all': [0.4, 0.6]}

In [60]:
from surprise.prediction_algorithms import SVD
from surprise.model_selection import GridSearchCV

param_grid = {'n_factors':[20, 50, 75, 100],'n_epochs': [5, 6, 7, 8, 9, 10, 11], 'lr_all': [0.002, .003, .004, 0.005],
               'reg_all': [.02, 0.4, .5, 0.6]}
gs_model = GridSearchCV(SVD,param_grid=param_grid,n_jobs = -1,joblib_verbose=5)
gs_model.fit(data)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    9.4s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   25.3s
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:   50.7s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done 866 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done 1136 tasks      | elapsed:  7.2min
[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed:  9.0min
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed: 11.6min
[Parallel(n_jobs=-1)]: Done 2162 tasks      | elapsed: 15.1min
[Parallel(n_jobs=-1)]: Done 2240 out of 2240 | elapsed: 16.0min finished


In [61]:
svd = SVD(n_factors=100, n_epochs=10, lr_all=0.005, reg_all=0.4)
svd.fit(trainset)
predictions = svd.test(testset)
print(accuracy.rmse(predictions))

RMSE: 1.1448
1.1447597198222013


### Predictions SVD

In [62]:
user_34_prediction = svd.predict('12', '85')
user_34_prediction

Prediction(uid='12', iid='85', r_ui=None, est=4.0, details={'was_impossible': False})

In [63]:
user_34_prediction[3]

4.0

In [64]:
def get_Iu(uid):
    try:
        return len(trainset.ur[trainset.to_inner_uid(uid)])
    except ValueError: # user was not part of the trainset
        return 0
def get_Ui(iid):
    """
args:
iid: the raw id of the item
returns:
the number of users that have rated the item.
"""
    try:
        return len(trainset.ir[trainset.to_inner_iid(iid)])
    except ValueError:
        return 0
df_predictions = pd.DataFrame(predictions, columns=['uid', 'iid', 'rui', 'est', 'details'])
df_predictions['Iu'] = df_predictions.uid.apply(get_Iu)
df_predictions['Ui'] = df_predictions.iid.apply(get_Ui)
df_predictions['err'] = abs(df_predictions.est - df_predictions.rui)

In [65]:
best_predictions = df_predictions.sort_values(by='err')[:10]
best_predictions

,uid,iid,rui,est,details,Iu,Ui,err
0,493,2167,4.0,4.0,{'was_impossible': False},48,43,0.0
4792,187,49932,4.0,4.0,{'was_impossible': False},200,5,0.0
13172,477,1701,4.0,4.0,{'was_impossible': False},482,4,0.0
4790,286,150,4.0,4.0,{'was_impossible': False},89,153,0.0
4789,405,5810,4.0,4.0,{'was_impossible': False},104,34,0.0
13180,4,2406,4.0,4.0,{'was_impossible': False},174,42,0.0
4786,20,1013,4.0,4.0,{'was_impossible': False},201,10,0.0
13183,379,480,4.0,4.0,{'was_impossible': False},32,189,0.0
13184,380,6373,4.0,4.0,{'was_impossible': False},988,59,0.0
13191,249,8368,4.0,4.0,{'was_impossible': False},843,79,0.0


### Graph Lab Model

In [66]:
# Make a train-test split
#train_data, validate_data = df.random_split(0.8)

### Cosine Similarity

In [67]:
from surprise import KNNBasic
from surprise.model_selection import cross_validate

In [68]:
sim_options = {
    'name': 'cosine',
    'user_based': False
}
 
knn = KNNBasic(sim_options=sim_options)

In [69]:
#knn.train(trainset)

### Surprise Baseline Only

In [70]:
from surprise import BaselineOnly

In [71]:
# showcasing the cross_validate function as well 
cross_validate(BaselineOnly(), data, verbose=True)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE, MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.1420  1.1597  1.1415  1.1518  1.1499  1.1490  0.0068  
MAE (testset)     0.8354  0.8470  0.8318  0.8399  0.8356  0.8379  0.0052  
Fit time          0.22    0.23    0.23    0.23    0.24    0.23    0.01    
Test time         0.08    0.37    0.08    0.07    0.08    0.13    0.12    


{'test_rmse': array([1.1420112 , 1.15971003, 1.14150367, 1.15184422, 1.14990514]),
 'test_mae': array([0.83536558, 0.84695153, 0.83179421, 0.83988939, 0.83563505]),
 'fit_time': (0.2186439037322998,
  0.23325705528259277,
  0.22863006591796875,
  0.23340487480163574,
  0.2395472526550293),
 'test_time': (0.0800011157989502,
  0.3669581413269043,
  0.0753030776977539,
  0.07332611083984375,
  0.07548689842224121)}

### KNN with GridSearch

In [72]:
from surprise import KNNWithMeans
from surprise.model_selection import GridSearchCV

In [73]:

sim_options = {
    "name": ["msd", "cosine"],
    "min_support": [2, 3, 4, 5],
    "user_based": [False, True],
}

param_grid = {"sim_options": sim_options}

gs = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse", "mae"], cv=3)
gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

### Grid Search

In [98]:
# param_grid = {
        #"n_epochs": [5, 10, 15, 20, 30, 40, 50, 100],
        # "lr_all": [0.001, 0.002, 0.005],
       # "reg_all": [0.02, 0.08, 0.4, 0.6]
#}

# smaller grid for testing
param_grid = {
    "n_epochs": [15, 20, 25],
    "lr_all": [0.002, 0.005],
    "reg_all": [0.02]
}
gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], refit=True, cv=5)

gs.fit(data)

training_parameters = gs.best_params["rmse"]

print("BEST RMSE: \t", gs.best_score["rmse"])
print("BEST MAE: \t", gs.best_score["mae"])
print("BEST params: \t", gs.best_params["rmse"])

BEST RMSE: 	 1.1494163879726262
BEST MAE: 	 0.8377630366977253
BEST params: 	 {'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.02}


In [75]:
from collections import defaultdict

from surprise import SVD
from surprise import Dataset


def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n


# First train an SVD algorithm on the movielens dataset.
#data = Dataset.load_builtin('ml-100k')
#trainset = df.build_full_trainset()
algo = SVD()
algo.fit(trainset)

# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=10)

# Print the recommended items for each user
#for uid, user_ratings in top_n.items():
    #print(uid, [iid for (iid, _) in user_ratings])

In [76]:
top_n('Toy Story 1995')

TypeError: 'collections.defaultdict' object is not callable

In [ ]:
get_top_n('Toy Story (1995)')

### Movie Reccomendation Model

In [77]:
df_2

,movieId,imdbId,tmdbId,title,userId_x,rating,userId_y,tag,timestamp,new_genres
0,1,114709,862.0,Toy Story (1995),1,4.0,336,pixar,1139045764,Adventure
0,1,114709,862.0,Toy Story (1995),1,4.0,336,pixar,1139045764,Animation
0,1,114709,862.0,Toy Story (1995),1,4.0,336,pixar,1139045764,Children
0,1,114709,862.0,Toy Story (1995),1,4.0,336,pixar,1139045764,Comedy
0,1,114709,862.0,Toy Story (1995),1,4.0,336,pixar,1139045764,Fantasy
...,...,...,...,...,...,...,...,...,...,...
233211,193565,1636780,71172.0,Gintama: The Movie (2010),184,3.5,184,gintama,1537098603,Sci-Fi
233212,193565,1636780,71172.0,Gintama: The Movie (2010),184,3.5,184,remaster,1537098592,Action
233212,193565,1636780,71172.0,Gintama: The Movie (2010),184,3.5,184,remaster,1537098592,Animation
233212,193565,1636780,71172.0,Gintama: The Movie (2010),184,3.5,184,remaster,1537098592,Comedy


In [78]:
df_2.head()

,movieId,imdbId,tmdbId,title,userId_x,rating,userId_y,tag,timestamp,new_genres
0,1,114709,862.0,Toy Story (1995),1,4.0,336,pixar,1139045764,Adventure
0,1,114709,862.0,Toy Story (1995),1,4.0,336,pixar,1139045764,Animation
0,1,114709,862.0,Toy Story (1995),1,4.0,336,pixar,1139045764,Children
0,1,114709,862.0,Toy Story (1995),1,4.0,336,pixar,1139045764,Comedy
0,1,114709,862.0,Toy Story (1995),1,4.0,336,pixar,1139045764,Fantasy


In [79]:
import RAKE
import pandas as pd
from rake_nltk import Rake
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

df_3 = df_2[['title','new_genres','tag','rating']]
df_3.head()

,title,new_genres,tag,rating
0,Toy Story (1995),Adventure,pixar,4.0
0,Toy Story (1995),Animation,pixar,4.0
0,Toy Story (1995),Children,pixar,4.0
0,Toy Story (1995),Comedy,pixar,4.0
0,Toy Story (1995),Fantasy,pixar,4.0


In [80]:
# initializing the new column
df_3['Key_words'] = ""

for index, row in df.iterrows():
    plot = row['tag']
    
    # instantiating Rake, by default it uses english stopwords from NLTK
    # and discards all puntuation characters as well
    r = Rake()

    # extracting the words by passing the text
    r.extract_keywords_from_text(plot)

    # getting the dictionary whith key words as keys and their scores as values
    key_words_dict_scores = r.get_word_degrees()
    
    # assigning the key words to the new column for the corresponding movie
    row['Key_words'] = list(key_words_dict_scores.keys())

# dropping the Plot column
df_3.drop(columns = ['tag'], inplace = True)

<ipython-input-80-4cbf3a1506ab>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_3['Key_words'] = ""


KeyError: 'tag'

In [81]:
# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(df['tag'])

# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)

KeyError: 'tag'

In [82]:
# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use in the function to match the indexes
indices = pd.Series(df.index)

#  defining the function that takes in movie title 
# as input and returns the top 10 recommended movies
def recommendations(title, cosine_sim = cosine_sim):
    
    # initializing the empty list of recommended movies
    recommended_movies = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_movies.append(list(df.index)[i])
        
    return recommended_movies

NameError: name 'cosine_sim' is not defined

In [83]:
recommendations('Toy Story (1995)')

NameError: name 'recommendations' is not defined

model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(movie_features_df_matrix)

### My Own Algorythem

In [84]:
ratings.head(1)

,userId,movieId,rating
0,1,1,4.0


In [85]:
from surprise import AlgoBase
from surprise.model_selection import cross_validate


class MyOwnAlgorithm(AlgoBase):

    def __init__(self):

        # Always call base method before doing anything.
        AlgoBase.__init__(self)

    def estimate(self, u, i):

        return 3


#data = Dataset.load_builtin('ml-100k')
algo = MyOwnAlgorithm()

cross_validate(algo, data, verbose=True)

Evaluating RMSE, MAE of algorithm MyOwnAlgorithm on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.1501  1.1663  1.1553  1.1558  1.1502  1.1555  0.0059  
MAE (testset)     0.8435  0.8510  0.8444  0.8452  0.8423  0.8453  0.0030  
Fit time          0.00    0.03    0.03    0.03    0.02    0.02    0.01    
Test time         0.09    0.05    0.05    0.07    0.06    0.06    0.01    


{'test_rmse': array([1.15006618, 1.16629944, 1.15534271, 1.15581472, 1.15016475]),
 'test_mae': array([0.84348969, 0.85096941, 0.84442406, 0.84516785, 0.84231666]),
 'fit_time': (2.002716064453125e-05,
  0.025764942169189453,
  0.026119232177734375,
  0.027784109115600586,
  0.018633127212524414),
 'test_time': (0.08844399452209473,
  0.05208587646484375,
  0.05065131187438965,
  0.06911206245422363,
  0.0550079345703125)}

In [86]:
class MyOwnAlgorithm(AlgoBase):

    def __init__(self):

        # Always call base method before doing anything.
        AlgoBase.__init__(self)

    def fit(self, trainset):

        # Here again: call base method before doing anything.
        AlgoBase.fit(self, trainset)

        # Compute the average rating. We might as well use the
        # trainset.global_mean attribute ;)
        self.the_mean = np.mean([r for (_, _, r) in
                                 self.trainset.all_ratings()])

        return self

    def estimate(self, u, i):

        return self.the_mean

In [87]:
    def estimate(self, u, i):

        sum_means = self.trainset.global_mean
        div = 1

        if self.trainset.knows_user(u):
            sum_means += np.mean([r for (_, r) in self.trainset.ur[u]])
            div += 1
        if self.trainset.knows_item(i):
            sum_means += np.mean([r for (_, r) in self.trainset.ir[i]])
            div += 1

        return sum_means / div

In [88]:
from surprise import PredictionImpossible

In [89]:
class MyOwnAlgorithm(AlgoBase):

    def __init__(self, sim_options={}, bsl_options={}):

        AlgoBase.__init__(self, sim_options=sim_options,
                          bsl_options=bsl_options)

    def fit(self, trainset):

        AlgoBase.fit(self, trainset)

        # Compute baselines and similarities
        self.bu, self.bi = self.compute_baselines()
        self.sim = self.compute_similarities()

        return self

    def estimate(self, u, i):

        if not (self.trainset.knows_user(u) and self.trainset.knows_item(i)):
            raise PredictionImpossible('User and/or item is unknown.')

        # Compute similarities between u and v, where v describes all other
        # users that have also rated item i.
        neighbors = [(v, self.sim[u, v]) for (v, r) in self.trainset.ir[i]]
        # Sort these neighbors by similarity
        neighbors = sorted(neighbors, key=lambda x: x[1], reverse=True)

        print('The 3 nearest neighbors of user', str(u), 'are:')
        for v, sim_uv in neighbors[:3]:
            print('user {0:} with sim {1:1.2f}'.format(v, sim_uv))

        # ... Aaaaand return the baseline estimate anyway ;)

In [90]:
import inspect

In [91]:
from collections import defaultdict

from surprise import SVD
from surprise import Dataset


def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for userId, movieId, rating in predictions.items():
        top_n[userId].append((movieId, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for userId, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[userId] = user_ratings[:n]

    return top_n


# First train an SVD algorithm on the movielens dataset.
#data = Dataset.load_builtin('ml-100k')
trainset = data.build_full_trainset()
algo = SVD()
algo.fit(trainset)

# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=3)

# Print the recommended items for each user
#for uid, user_ratings in top_n.items():
   # print(uid, [iid for (iid, _) in user_ratings])

AttributeError: 'list' object has no attribute 'items'

In [92]:
get_top_n('Toy Story (1995)')

AttributeError: 'str' object has no attribute 'items'

In [93]:
ratings

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100831,610,166534,4.0
100832,610,168248,5.0
100833,610,168250,5.0
100834,610,168252,5.0


In [94]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [95]:
#get_top_n('Toy Story (1995)')

In [96]:
from collections import defaultdict

from surprise import SVD
from surprise import Dataset


def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    #top_n = defaultdict(list)
    #for uid, iid, true_r, est, _ in predictions:
        #top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n


# First train an SVD algorithm on the movielens dataset.
#data = Dataset.load_builtin('ml-100k')
trainset = data.build_full_trainset()
algo = SVD()
algo.fit(trainset)

# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=10)

# Print the recommended items for each user
#for uid, user_ratings in top_n.items():
   # print(uid, [iid for (iid, _) in user_ratings])

In [97]:
get_top_n('Toy Story (1995)')

defaultdict(list,
            {226: [(904, 4.516344761199507),
              (318, 4.490506512380902),
              (858, 4.395793990061614),
              (1201, 4.391977787629009),
              (750, 4.382704750356151),
              (933, 4.380396559112212),
              (1196, 4.373662961693453),
              (1250, 4.364263440927925),
              (2324, 4.347662213874222),
              (50, 4.337102718857136)],
             523: [(1221, 5.0),
              (2502, 5.0),
              (2858, 5.0),
              (1267, 5.0),
              (78499, 5.0),
              (47, 5.0),
              (2329, 5.0),
              (593, 5.0),
              (475, 5.0),
              (7090, 5.0)],
             354: [(1233, 4.436244505521871),
              (2959, 4.427367694836075),
              (898, 4.403493185166812),
              (2571, 4.390820197698904),
              (1172, 4.359175041466265),
              (1104, 4.357064143924769),
              (1266, 4.3549246558849894),
        